# Imports, hyperparameters and functions

### Imports

In [35]:
from __future__ import print_function
from termcolor import colored,cprint # enable color in print statements

import keras
import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import objectives, backend as K
from keras.datasets import mnist

import numpy as np

In [36]:
print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)

Keras version:  2.2.4
Tensorflow version:  1.13.1


### Hyperparameters

In [6]:
# Set the hyperparameters
#------------------------------
batch_size = 100
original_dim = 28 * 28 # original dimension of an image (height x width)
latent_dim = 2
intermediate_dim = 256
no_epochs = 5
epsilon_std = 1.0

### Functions

In [37]:
def sampling(args: tuple):
    # we grab the variables from the tuple
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# Variational Autoencoders

In [38]:
# DEFINE THE ENCODER
#------------------------------
# Input to the encoder
x = Input(shape = (original_dim, ), name = 'input')

# Intermediate layer
h = Dense(intermediate_dim, activation = 'relu', name = 'encoding')(x)

# Define the mean of the latent space
z_mean = Dense(latent_dim, name = 'mean')(h)

# Define the log variance of the latent space
z_log_var = Dense(latent_dim, name = 'log-variance')(h)

z = Lambda(sampling, output_shape = (latent_dim, ))([z_mean, z_log_var])

# Encoder
encoder = Model(x, [z_mean, z_log_var], name = 'encoder')
print(colored('ENCODER', 'red'))
encoder.summary()

ENCODER
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 784)          0                                            
__________________________________________________________________________________________________
encoding (Dense)                (None, 256)          200960      input[0][0]                      
__________________________________________________________________________________________________
mean (Dense)                    (None, 2)            514         encoding[0][0]                   
__________________________________________________________________________________________________
log-variance (Dense)            (None, 2)            514         encoding[0][0]                   
Total params: 201,988
Trainable params: 201,988
Non-trainable params: 0
_____________________________

In [39]:
# DEFINE THE DECODER
#------------------------------
# Input to the decoder
input_decoder = Input(shape = (latent_dim, ), name = 'decoder_input')

# Takes the latent space to the intermediate dimension
decoder_h = Dense(intermediate_dim, activation = 'relu', name = 'decoder_h')(input_decoder)

# Gets the mean from the original dimension
x_decoded = Dense(original_dim, activation = 'sigmoid', name = 'flat_decoded')(decoder_h)

# Decoder
decoder = Model(input_decoder, x_decoded, name = 'decoder')
print(colored('DECODER', 'blue'))
decoder.summary()

DECODER
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
decoder_h (Dense)            (None, 256)               768       
_________________________________________________________________
flat_decoded (Dense)         (None, 784)               201488    
Total params: 202,256
Trainable params: 202,256
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Combine encoder and decoder into a single VAE model
#------------------------------
# grab the output. Recall, that we need to grab the 3rd element our sampling z
output_combined = decoder(encoder(x)[1])

# link the input and the overall output
vae = Model(x, output_combined)

# print out what the overall model looks like
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 784)               0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2)]    201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________
